# Analyse des schémas de comportement des utilisateurs dans les applications DeFI
## Analyse des API Web 3 en vue de constituer un jeu de données

### API KEYS

In [33]:
from dotenv import load_dotenv
import os 

load_dotenv()
cg_api_key = os.getenv('CG_API_KEY')
eth_api_key = os.getenv('ETH_API_KEY')


### PROTOCOLS

In [ ]:
import requests
import json
import time

KEY_PROTOCOLS = [                                   # Liste des protocoles clés et leurs identifiants sur CoinGecko
    "uniswap",          # DEX
    "curve-dao-token",  # DEX 
    "balancer",         # DEX
    "aave",             # Lending
    "maker",            # Lending
    "yearn-finance",    # Yield Farming
    "harvest-finance",  # Yield Farming
    "dai",              # Stablecoin
    "usd-coin",         # Stablecoin
    "tether",           # Stablecoin
    "nftfi",            # NFT-Fi (optionnel)
]

def get_defi_protocol_info(protocol_id):
    """
    Récupère les informations détaillées d'un protocole spécifique.
    """
    url = f"https://api.coingecko.com/api/v3/coins/{protocol_id}"
    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": cg_api_key                 # Clé API CoinGecko
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            platforms = data.get('platforms', {})

            protocol_info = {
                'name': data.get('name', 'N/A'),
                'type': infer_protocol_type(protocol_id),           # Déduction du type selon la liste fournie
                'blockchain_contracts': [
                    {'blockchain': blockchain, 'contract': contract}
                    for blockchain, contract in platforms.items()
                ],
                'website_url': data.get('links', {}).get('homepage', ['N/A'])[0],           # URL officielle
                'symbol': data.get('symbol', 'N/A'),                                        # Symbole du token associé
                'market_cap_rank': data.get('market_cap_rank', 'N/A'),                      # Classement par capitalisation
                'description': data.get('description', {}).get('en', '')[:250],             # Description limitée à 250 caractères
            }
            return protocol_info
        
        elif response.status_code == 404:
            print(f"Protocole introuvable : {protocol_id}")
        elif response.status_code == 429:
            print("Trop de requêtes. Attente de 10 secondes...")
            time.sleep(10)                                              # Attendre 10 secondes en cas de limite d'API
            return get_defi_protocol_info(protocol_id)                  # Réessayer
        else:
            print(f"Erreur inconnue avec {protocol_id}: {response.status_code}")
    except Exception as e:
        print(f"Erreur avec {protocol_id} : {str(e)}")
    return None

def infer_protocol_type(protocol_id):
    """
    Déduit le type du protocole en fonction de son identifiant.
    """
    if protocol_id in ["uniswap", "sushiswap", "curve-dao-token", "balancer"]:
        return "DEX"
    elif protocol_id in ["aave", "compound", "maker"]:
        return "Lending"
    elif protocol_id in ["yearn-finance", "harvest-finance", "curve-dao-token"]:
        return "Yield Farming"
    elif protocol_id in ["dai", "usd-coin", "tether"]:
        return "Stablecoin"
    elif protocol_id in ["nftfi", "nifty-gateway", "opensea"]:
        return "NFT-Fi"
    else:
        return "DeFi"

def save_protocols_to_json(protocols, filename):
    """
    Sauvegarde une liste de protocoles dans un fichier JSON.
    """
    with open(filename, 'w') as f:
        json.dump(protocols, f, indent=4)


if __name__ == "__main__":                                                  # Exécuter le script                  
    print("Extraction des données pour les protocoles clés...")
    protocols_data = []
    for protocol_id in KEY_PROTOCOLS:
        protocol_info = get_defi_protocol_info(protocol_id)
        if protocol_info:
            protocols_data.append(protocol_info)
                                                                            # Respecter la limite de requêtes : attendre 2 secondes entre chaque requête
            time.sleep(2)

    print(f"{len(protocols_data)} protocoles récupérés.")
    
    save_protocols_to_json(protocols_data, 'key_protocols.json')        # Sauvegarder dans un fichier JSON
    print("Données sauvegardées dans 'key_protocols.json'.")

Extraction des données pour les protocoles clés...
11 protocoles récupérés.
Données sauvegardées dans 'key_protocols.json'.


### CONTRACTS

In [20]:
import json
import pandas as pd

with open('key_protocols.json', 'r') as f:
    protocols_data = json.load(f)

data = []

for protocol in protocols_data:                                         # Parcours des protocoles pour extraire les contrats          
    blockchain_contracts = protocol.get("blockchain_contracts", [])
    
    for contract_info in blockchain_contracts:                          # Parcours des contrats de chaque protocole
        contract_address = contract_info.get("contract")
        blockchain = contract_info.get("blockchain").lower()  
        
        data.append({
            "protocol": protocol.get("name"),
            "blockchain": blockchain,
            "contract": contract_address
        })

df = pd.DataFrame(data)
df.to_csv('contracts_data.csv', index=False)

print("Les données ont été enregistrées dans 'contracts_data.csv'.")

blockchain_counts = df['blockchain'].value_counts()

print("Nombre de contrats associés à chaque blockchain :")
for blockchain, count in blockchain_counts.items():
    print(f"- {blockchain.capitalize()}: {count} contrats")

Les données ont été enregistrées dans 'contracts_data.csv'.
Nombre de contrats associés à chaque blockchain :
- Ethereum: 11 contrats
- Avalanche: 7 contrats
- Energi: 7 contrats
- Polygon-pos: 7 contrats
- Near-protocol: 6 contrats
- Optimistic-ethereum: 6 contrats
- Arbitrum-one: 6 contrats
- Sora: 5 contrats
- Harmony-shard-0: 4 contrats
- Huobi-token: 4 contrats
- Binance-smart-chain: 3 contrats
- Fantom: 3 contrats
- Xdai: 3 contrats
- Base: 3 contrats
- Celo: 2 contrats
- Tron: 2 contrats
- Solana: 2 contrats
- Aptos: 2 contrats
- Algorand: 1 contrats
- Kava: 1 contrats
- Stellar: 1 contrats
- Polygon-zkevm: 1 contrats
- Sui: 1 contrats
- Hedera-hashgraph: 1 contrats
- Polkadot: 1 contrats
- Zksync: 1 contrats
- The-open-network: 1 contrats


In [36]:
import requests
import pandas as pd
from datetime import datetime, timezone
import time

# Chargement du CSV avec les contrats Ethereum
try:
    df = pd.read_csv('contracts_data.csv')
    print("Fichier CSV chargé avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV: {e}")
    raise

# Filtrage des contrats Ethereum
ethereum_contracts = df[df['blockchain'] == 'ethereum']
print(f"Nombre de contrats Ethereum trouvés: {len(ethereum_contracts)}")

# Liste pour stocker les données des utilisateurs
user_data = []
block_data = {}  # Dictionnaire pour stocker les blocs par contrat

# Set pour éviter les doublons de logs
seen_logs = set()

# Fonction pour récupérer les logs d'un contrat avec pagination
def get_contract_logs(contract_address, start_block, end_block):
    print(f"Récupération des logs pour le contrat {contract_address} entre les blocs {start_block} et {end_block}.")
    url = f'https://api.etherscan.io/api'
    params = {
        'module': 'logs',
        'action': 'getLogs',
        'fromBlock': start_block,
        'toBlock': end_block,
        'address': contract_address,
        'apikey': eth_api_key,
        'page': 1,  # On commence par la première page
        'offset': 1000  # Limité à 1000 logs par page
    }
    
    all_logs = []  # Liste pour stocker tous les logs récupérés
    while True:
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Vérifie si la réponse est réussie (status code 200)
            data = response.json()
            
            if data.get('status') == '1':  # Si l'appel API a réussi
                logs = data.get('result', [])
                all_logs.extend(logs)  # Ajouter les logs de cette page à la liste
                
                if len(logs) < 1000:  # Si moins de 1000 logs, on a récupéré tous les logs
                    break
                else:
                    params['page'] += 1  # Passer à la page suivante
                    time.sleep(0.2)  # Respecter la limite de 5 appels par seconde (1 appel toutes les 200ms)
            else:
                print(f"Erreur lors de la récupération des logs: {data}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de l'appel API pour récupérer les logs du contrat {contract_address}: {e}")
            break
    
    return all_logs

# Fonction pour convertir un timestamp Unix en date formatée
def convert_timestamp_to_date(timestamp):
    try:
        timestamp = int(timestamp, 16)  # Conversion d'un timestamp hexadécimal en entier
        return datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')  # Utilisation de timezone.utc
    except ValueError as e:
        print(f"Erreur lors de la conversion du timestamp: {e}")
        return None

# Fonction pour récupérer les blocs correspondant à une période
def get_block_range_for_date_range(start_date, end_date):
    print(f"Calcul des blocs pour la période {start_date} - {end_date}.")
    # Convertir les dates en timestamps Unix
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    end_timestamp = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())
    
    # Affichage des timestamps pour débogage
    print(f"Start timestamp: {start_timestamp}, End timestamp: {end_timestamp}")
    
    # Calculer les blocs en fonction des timestamps (Etherscan API)
    start_block_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={start_timestamp}&closest=before&apikey={eth_api_key}'
    end_block_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={end_timestamp}&closest=after&apikey={eth_api_key}'

    try:
        start_block_response = requests.get(start_block_url).json()
        end_block_response = requests.get(end_block_url).json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des blocs: {e}")
        return None, None

    if start_block_response.get('status') == '1' and end_block_response.get('status') == '1':
        start_block = int(start_block_response['result'])
        end_block = int(end_block_response['result'])
        print(f"Blocs trouvés: start_block = {start_block}, end_block = {end_block}")
        return start_block, end_block
    else:
        print(f"Erreur lors de la récupération des blocs. Start response: {start_block_response}, End response: {end_block_response}")
        return None, None

# Calcul des blocs pour la période de décembre 2024 une seule fois
start_date = '2024-12-01'
end_date = '2024-12-31'
start_block, end_block = get_block_range_for_date_range(start_date, end_date)

# Vérification que les blocs ont bien été récupérés
if not start_block or not end_block:
    print("Erreur : Impossible de récupérer les blocs pour la période spécifiée.")
    exit(1)

# Sauvegarde des numéros de blocs dans le dictionnaire pour chaque contrat
block_data['start_block'] = start_block
block_data['end_block'] = end_block

# Parcours des contrats Ethereum pour extraire les logs du mois de décembre 2024
for _, row in ethereum_contracts.iterrows():
    contract_address = row['contract']
    protocol_name = row['protocol']  # Le nom du protocole est extrait ici
    
    # Récupérer les logs pour ce contrat avec pagination
    logs = get_contract_logs(contract_address, start_block, end_block)

    if logs:  # Vérifie si des logs ont été récupérés
        print(f"Logs récupérés pour le contrat {contract_address}, nombre de logs: {len(logs)}.")
        for log in logs:
            timestamp = log.get('timeStamp', None)
            if timestamp:
                created_at = convert_timestamp_to_date(timestamp)
                if created_at:
                    user_data.append({
                        'user_id': log['address'],  # Adresse de l'utilisateur
                        'address': contract_address,  # Adresse du contrat
                        'blockchain': 'ethereum',  # Blockchain associée
                        'protocol': protocol_name,  # Nom du protocole associé
                        'created_at': created_at  # Date formatée
                    })
    else:
        print(f"Aucun log trouvé pour le contrat {contract_address}.")

# Création d'un DataFrame avec les données récupérées
user_df = pd.DataFrame(user_data)
print(f"Nombre de lignes dans le DataFrame avant suppression des doublons: {len(user_df)}")

# Suppression des doublons dans le DataFrame basé sur les colonnes 'user_id' et 'address'
user_df = user_df.drop_duplicates(subset=['user_id', 'address'])

print(f"Nombre de lignes après suppression des doublons: {len(user_df)}")

# Sauvegarde des données des utilisateurs dans un fichier CSV
try:
    user_df.to_csv('user_contracts_december_2024.csv', index=False)
    print("Les données des utilisateurs ont été enregistrées dans 'user_contracts_december_2024.csv'.")
except Exception as e:
    print(f"Erreur lors de la sauvegarde des données dans le fichier CSV: {e}")

Fichier CSV chargé avec succès.
Nombre de contrats Ethereum trouvés: 11
Calcul des blocs pour la période 2024-12-01 - 2024-12-31.
Start timestamp: 1733007600, End timestamp: 1735599600
Blocs trouvés: start_block = 21303634, end_block = 21518436
Récupération des logs pour le contrat 0x1f9840a85d5af5bf1d1762f925bdaddc4201f984 entre les blocs 21303634 et 21518436.
Erreur lors de la récupération des logs: {'status': '0', 'message': 'Result window is too large, PageNo x Offset size must be less than or equal to 10000', 'result': None}
Logs récupérés pour le contrat 0x1f9840a85d5af5bf1d1762f925bdaddc4201f984, nombre de logs: 10000.
Récupération des logs pour le contrat 0xd533a949740bb3306d119cc777fa900ba034cd52 entre les blocs 21303634 et 21518436.
Erreur lors de la récupération des logs: {'status': '0', 'message': 'Result window is too large, PageNo x Offset size must be less than or equal to 10000', 'result': None}
Logs récupérés pour le contrat 0xd533a949740bb3306d119cc777fa900ba034cd52, 